In [1]:
from google.colab import drive
drive.mount('/gdrive')
% ls
% cd /gdrive/My Drive/CS470/StarGAN-Voice-Conversion-master

Mounted at /gdrive
sample_data/
/gdrive/My Drive/CS470/StarGAN-Voice-Conversion-master


In [2]:
pip install pyworld

     |████████████████████████████████| 225kB 5.4MB/s 
  Created wheel for pyworld: filename=pyworld-0.2.12-cp36-cp36m-linux_x86_64.whl size=609677 sha256=2ad77b10fd6e77a3eec70902c0ed49d54ae8ac8fc6481579b758c574aa17ffb5
  Stored in directory: /root/.cache/pip/wheels/d0/e4/1c/a508000462b83164d5eba9a4b46f39b4b1645ac952bbe71551
Successfully built pyworld


In [13]:
pip install sox

In [12]:
pip install git+git://github.com/pytorch/audio

  Cloning git://github.com/pytorch/audio to /tmp/pip-req-build-7ohwyk40
  Running command git clone -q git://github.com/pytorch/audio /tmp/pip-req-build-7ohwyk40
  ERROR: Failed building wheel for torchaudio
  Running setup.py clean for torchaudio
Failed to build torchaudio
  Found existing installation: torchaudio 0.7.0
    Uninstalling torchaudio-0.7.0:
      Successfully uninstalled torchaudio-0.7.0
    Running setup.py install for torchaudio ... error
  Rolling back uninstall of torchaudio
  Moving to /usr/local/lib/python3.6/dist-packages/torchaudio-0.7.0.dist-info/
   from /usr/local/lib/python3.6/dist-packages/~orchaudio-0.7.0.dist-info
  Moving to /usr/local/lib/python3.6/dist-packages/torchaudio/
   from /usr/local/lib/python3.6/dist-packages/~orchaudio
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-req-build-7ohwyk40/setup.py'"'"'; __file__='"'"'/tmp/pip-req-build-7ohwyk40/setup.py'"'"';f=ge

In [11]:
pip install torchaudio

     |████████████████████████████████| 7.6MB 5.4MB/s 


In [15]:
!apt -qq install -y sox

The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 14 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack .../0-libopencore-amrnb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack .../1-libopencore-amrwb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrwb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected pa

In [17]:
!apt-get install libsox-fmt-all libsox-dev sox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
The following additional packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libsox-fmt-ao libsox-fmt-mp3
  libsox-fmt-oss libsox-fmt-pulse
Suggested packages:
  libaudio2
The following NEW packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libsox-dev libsox-fmt-all
  libsox-fmt-ao libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse
0 upgraded, 10 newly installed, 0 to remove and 14 not upgraded.
Need to get 507 kB of archives.
After this operation, 2,435 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libao-common all 1.2.2+20180113-1ubuntu1 [6,644 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libao4 amd64 1.2.2+20180113-1ubuntu1 [35.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libid3tag0 amd64 0.15.1b-13 [31.2 kB]


In [20]:
import librosa
import numpy as np
import os, sys
import argparse
import pyworld
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from utils import *
from tqdm import tqdm
from collections import defaultdict
from collections import namedtuple
from sklearn.model_selection import train_test_split
import glob
from os.path import join, basename
import subprocess

def resample(spk, origin_wavpath, target_wavpath):
    wavfiles = [i for i in os.listdir(join(origin_wavpath, spk)) if i.endswith(".wav")]
    for wav in wavfiles:
        folder_to = join(target_wavpath, spk)
        os.makedirs(folder_to, exist_ok=True)
        wav_to = join(folder_to, wav)
        wav_from = join(origin_wavpath, spk, wav)
        subprocess.call(['sox', wav_from, "-r", "16000", wav_to])
    return 0

def resample_to_16k(origin_wavpath, target_wavpath, num_workers=1):
    os.makedirs(target_wavpath, exist_ok=True)
    spk_folders = os.listdir(origin_wavpath)
    print(f"> Using {num_workers} workers!")
    executor = ProcessPoolExecutor(max_workers=num_workers)
    futures = []
    for spk in spk_folders:
        futures.append(executor.submit(partial(resample, spk, origin_wavpath, target_wavpath)))
    result_list = [future.result() for future in tqdm(futures)]
    print(result_list)

def split_data(paths):
    indices = np.arange(len(paths))
    test_size = 0.1
    train_indices, test_indices = train_test_split(indices, test_size=test_size, random_state=1234)
    train_paths = list(np.array(paths)[train_indices])
    test_paths = list(np.array(paths)[test_indices])
    return train_paths, test_paths

def get_spk_world_feats(spk_fold_path, mc_dir_train, mc_dir_test, sample_rate=16000):
    paths = glob.glob(join(spk_fold_path, '*.wav'))
    spk_name = basename(spk_fold_path)
    train_paths, test_paths = split_data(paths)
    f0s = []
    coded_sps = []
    for wav_file in train_paths:
        f0, _, _, _, coded_sp = world_encode_wav(wav_file, fs=sample_rate)
        f0s.append(f0)
        coded_sps.append(coded_sp)
    log_f0s_mean, log_f0s_std = logf0_statistics(f0s)
    coded_sps_mean, coded_sps_std = coded_sp_statistics(coded_sps)
    np.savez(join(mc_dir_train, spk_name+'_stats.npz'), 
            log_f0s_mean=log_f0s_mean,
            log_f0s_std=log_f0s_std,
            coded_sps_mean=coded_sps_mean,
            coded_sps_std=coded_sps_std)
    
    for wav_file in tqdm(train_paths):
        wav_nam = basename(wav_file)
        f0, timeaxis, sp, ap, coded_sp = world_encode_wav(wav_file, fs=sample_rate)
        normed_coded_sp = normalize_coded_sp(coded_sp, coded_sps_mean, coded_sps_std)
        np.save(join(mc_dir_train, wav_nam.replace('.wav', '.npy')), normed_coded_sp, allow_pickle=False)
    
    for wav_file in tqdm(test_paths):
        wav_nam = basename(wav_file)
        f0, timeaxis, sp, ap, coded_sp = world_encode_wav(wav_file, fs=sample_rate)
        normed_coded_sp = normalize_coded_sp(coded_sp, coded_sps_mean, coded_sps_std)
        np.save(join(mc_dir_test, wav_nam.replace('.wav', '.npy')), normed_coded_sp, allow_pickle=False)
    return 0


if __name__ == '__main__':
    # parser = argparse.ArgumentParser()


    # sample_rate_default = 16000
    # origin_wavpath_default = "./data/VCTK-Corpus/wav48"
    # target_wavpath_default = "./data/VCTK-Corpus/wav16"
    # mc_dir_train_default = './data/mc/train'
    # mc_dir_test_default = './data/mc/test'

    # parser.add_argument("--sample_rate", type = int, default = 16000, help = "Sample rate.")
    # parser.add_argument("--origin_wavpath", type = str, default = origin_wavpath_default, help = "The original wav path to resample.")
    # parser.add_argument("--target_wavpath", type = str, default = target_wavpath_default, help = "The original wav path to resample.")
    # parser.add_argument("--mc_dir_train", type = str, default = mc_dir_train_default, help = "The directory to store the training features.")
    # parser.add_argument("--mc_dir_test", type = str, default = mc_dir_test_default, help = "The directory to store the testing features.")
    # parser.add_argument("--num_workers", type = int, default = None, help = "The number of cpus to use.")

    # argv = parser.parse_args()

    # sample_rate = argv.sample_rate
    # origin_wavpath = argv.origin_wavpath
    # target_wavpath = argv.target_wavpath
    # mc_dir_train = argv.mc_dir_train
    # mc_dir_test = argv.mc_dir_test
    # num_workers = argv.num_workers if argv.num_workers is not None else cpu_count()

    sample_rate = 16000
    origin_wavpath = "./data/VCTK-Corpus/wav48"
    target_wavpath = "./data/VCTK-Corpus/wav16"
    mc_dir_train = './data/mc/train'
    mc_dir_test = './data/mc/test'
    num_workers = cpu_count()

    # The original wav in VCTK is 48K, first we want to resample to 16K
    resample_to_16k(origin_wavpath, target_wavpath, num_workers=num_workers)

    # WE only use 10 speakers listed below for this experiment.
    speaker_used = ['262', '272', '229', '232', '292', '293', '360', '361', '248', '251']
    speaker_used = ['p'+i for i in speaker_used]

    ## Next we are to extract the acoustic features (MCEPs, lf0) and compute the corresponding stats (means, stds). 
    # Make dirs to contain the MCEPs
    os.makedirs(mc_dir_train, exist_ok=True)
    os.makedirs(mc_dir_test, exist_ok=True)

    num_workers = len(speaker_used) #cpu_count()
    print("number of workers: ", num_workers)
    executor = ProcessPoolExecutor(max_workers=num_workers)

    work_dir = target_wavpath
    # spk_folders = os.listdir(work_dir)
    # print("processing {} speaker folders".format(len(spk_folders)))
    # print(spk_folders)

    futures = []
    for spk in speaker_used:
        spk_path = os.path.join(work_dir, spk)
        futures.append(executor.submit(partial(get_spk_world_feats, spk_path, mc_dir_train, mc_dir_test, sample_rate)))
    result_list = [future.result() for future in tqdm(futures)]
    print(result_list)
    sys.exit(0)



> Using 2 workers!






  0%|          | 0/109 [00:00<?, ?it/s]



  1%|          | 1/109 [00:21<39:12, 21.78s/it]Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
_pickle.PicklingError: Can't pickle <function resample at 0x7f6a08f04378>: it's not the same object as __main__.resample
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/usr/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
_pickle.PicklingError: Can't pickle <function resample at 0x7f6a08f04378>: it's not the same object as __main__.resample




  2%|▏         | 2/109 [01:43<1:10:49, 39.71s/it]Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 234, i

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
number of workers:  10


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.



 75%|███████▌  | 279/370 [58:37<18:30, 12.21s/it]



 87%|████████▋ | 284/327 [1:05:33<08:33, 11.95s/it]



 96%|█████████▌| 364/380 [1:07:54<02:40, 10.05s/it]



 67%|██████▋   | 238/354 [56:54<35:28, 18.35s/it]



 91%|█████████ | 309/341 [1:04:51<06:00, 11.26s/it]



 78%|███████▊  | 286/369 [1:00:40<15:20, 11.09s/it]



 49%|████▉     | 175/356 [46:12<38:13, 12.67s/it]



 76%|███████▋  | 258/338 [58:33<16:52, 12.66s/it]



 72%|███████▏  | 267/369 [58:17<21:15, 12.51s/it]



 81%|████████  | 307/381 [1:02:00<16:57, 13.74s/it]



 96%|█████████▌| 365/380 [1:08:06<02:38, 10.58s/it]



 87%|████████▋ | 285/327 [1:05:46<08:37, 12.31s/it]



 68%|██████▊   | 239/354 [57:07<32:15, 16.83s/it]



 91%|█████████ | 310/341 [1:05:03<05:57, 11.54s/it]



 78%|███████▊  | 287/369 [1:00:53<16:01, 11.72s/it]



 76%|███████▌  | 280/370 [58:59<22:18, 14.87s/it]



 77%|███████▋  | 259/338 [58:48<17:17, 13.13s/it]



 49%|████▉     | 176/356 [46:27<39:44, 13.

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
